In [21]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Conv1D, Dense, LSTM, Dropout, MaxPooling1D, Bidirectional, Concatenate, GlobalAveragePooling1D, SpatialDropout1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Activation
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
import tensorflow as tf

In [22]:
url_data = 'https://raw.githubusercontent.com/TeodorRusKvi/Tekstanalyse/main/git_NLP_data/'

# Last inn 'X_train_LSTM' fra en CSV-fil
X_df = pd.read_csv(url_data+'X_tensorflow.csv')
# Konverter hele DataFrame til et NumPy array
X = X_df.to_numpy()

# Last inn 'y_train_LSTM' fra en CSV-fil
y_df = pd.read_csv(url_data+'y_liberal.csv')
# Konverter hele DataFrame til et NumPy array
y = y_df.to_numpy()

# Last inn 'y_train_LSTM' fra en CSV-fil
embeddings_GloVe = pd.read_csv(url_data+'embeddings_glove.csv')
# Konverter hele DataFrame til et NumPy array
embeddings_GloVe = embeddings_GloVe.to_numpy()


# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

In [38]:
tf.random.set_seed(42)

#batch_size = None
#batch_size=64

def model_build():
    model = Sequential()
    model.add(Input(shape=(20,)))
    # Adding the Embedding layer with pre-trained weights and specifying input_length
    model.add(Embedding(input_dim=embeddings_GloVe.shape[0],
                        output_dim=embeddings_GloVe.shape[1],
                        weights=[embeddings_GloVe],
                        trainable=False))

    model.add(SpatialDropout1D(0.2))
    model.add(Conv1D(filters=64, kernel_size=1, activation='relu'))
    model.add(Dense(100, activation='relu'))
    # It's common to add a MaxPooling layer after a Conv layer to reduce dimensionality
    model.add(MaxPooling1D())

    model.add(Bidirectional(LSTM(64, # Number of hidden states, number of reccurent units for each vector
        activation='tanh',
        recurrent_activation='sigmoid',
        recurrent_dropout=0.2,
        return_sequences=False)))

    model.add(Dense(77, activation='relu'))
    model.add(Dropout(0.20))
    model.add(Dense(num_classes, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Model summary
    model.summary()
    return model

early_stopping_callback = EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)

In [39]:
input_length = 20
num_classes = 1
EPOCHS=20
BATCH_SIZE = 32

model = model_build()

early_stopping_callback = EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)


history = model.fit(X_train, y_train,
                    epochs=20,
                    validation_data=(X_val, y_val),
                    batch_size=BATCH_SIZE,
                    callbacks=early_stopping_callback)

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_12 (Embedding)    (None, 20, 100)           2223500   
                                                                 
 spatial_dropout1d_12 (Spat  (None, 20, 100)           0         
 ialDropout1D)                                                   
                                                                 
 conv1d_12 (Conv1D)          (None, 20, 64)            6464      
                                                                 
 dense_18 (Dense)            (None, 20, 100)           6500      
                                                                 
 max_pooling1d_9 (MaxPoolin  (None, 10, 100)           0         
 g1D)                                                            
                                                                 
 bidirectional_9 (Bidirecti  (None, 128)              

In [37]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, batch_size=BATCH_SIZE)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

61/61 [==============================] - 0s 7ms/step - loss: 0.5090 - accuracy: 0.7631
Test Loss: 0.50902259349823
Test Accuracy: 0.763089656829834
